In [4]:
import pandas as pd
import numpy as np
import time
import re
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer, util
from transliterate import translit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import Normalizer
import gc

## Кейс по нечеткому сравнению наименований шин между выгрузками из разных программ с целью подтягивания кодов из 1C

# Алтухов Николай

Загрузка файла выгрузки, с которым будут сопоставляться наименования из выгрузки из 1C

In [2]:
df = pd.read_csv('Gr_Krasn.csv',encoding='Windows-1251', low_memory=False)

In [3]:
df

,Наименование
0,R13 155/70 RoyalBlack Royal ECO
1,R13 155/70 RoyalBlack ROYALMILE
2,R13 155/70 Sailun Atrezzo Eco
3,R13 155/70 Kapsen H202 ComfortMax A/S
4,R13 155/70 Kapsen HD918
...,...
2381,R21 315/40 ILINK SnowGripper 2
2382,R21 315/40 Arivo Winmaster ProX ARW5
2383,R22 275/40 ILINK SnowGripper 2
2384,R22 275/40 ILINK WinterVorhut Stud 2


Загрузка выгрузки из 1C

In [4]:
df_1C = pd.read_excel('Данные из 1C.xlsx')
df_1C

,Наименование,Код
0,R13 155/70 75T ROYAL BLACK ROYAL MILE,ЦБ-00007482
1,R13 155/70 75T Sailun ATREZZO ECO,ЦБ-00006440
2,R13 155/70 75T KAPSEN H202 ComfortMax A/S,ЦБ-00007311
3,R13 155/70 RoyalBlack Royal A/S,ЦБ-00005134
4,R13 155/70 RoyalBlack Royal Mile,00-00003373
...,...,...
4883,R215/60 Автошина Yokohama 99T XL iceGuard Stud...,ЦБ-00006877
4884,R22 275/40 ILINK SNOWGRIPPER II 107H XL,ЦБ-00007606
4885,R22 275/40 ILINK WINTERVORHUT STUD Ⅱ 107T XL,ЦБ-00007701
4886,R22 285/40 Автошина Nokian Tyres T 110 HKPL 9 ...,ЦБ-00006836


Функции для очистки наименований

In [5]:
def remove_words_containing(text, substrings):
    # Регулярное выражение для поиска слов, содержащих подстроки
    pattern = r'\b\w*(' + '|'.join([substrings]) + r')\w*\b'
    # Замена слов, содержащих подстроки, на пустую строку
    return re.sub(pattern, '', text)
    
def process_column(df, column_name, new_column_name, substrings_to_remove="шин|овая"):
    # Приведение текста к нижнему регистру
    df[new_column_name] = df[column_name].str.lower()
    # Замена буквы 'ё' на 'е'
    df[new_column_name] = df[new_column_name].str.replace('ё', 'е')

    # Замена символов 'ⅱ' и 'Ⅱ' на 'ii'
    df[new_column_name] = df[new_column_name].str.replace(r'ⅱ|Ⅱ', 'ii', regex=True)

    # Удаление апострофов и скобок
    df[new_column_name] = df[new_column_name].str.replace(r"\(|\)|\'", '', regex=True)
    # Замена всех пробельных символов на пробелы, а также дефиса и нижнего подчеркивания
    df[new_column_name] = df[new_column_name].str.replace(r'\s+|\-|\_',' ', regex=True)
    # Удаление запятых и точек, кроме случаев, когда они находятся в слове или числе (например, запятая останется в значении "8,25")
    df[new_column_name] = df[new_column_name].str.replace(r"(?<!\w),|,(?!\w)|(?<!\w)\.|\.(?!\w)", '', regex=True)
    # Замена запятых на точки
    df[new_column_name] = df[new_column_name].str.replace(r",", '.', regex=True)

    # Удаление слов шина, автошина, легковая, грузовая
    df[new_column_name] = df[new_column_name].apply(lambda x: remove_words_containing(x, substrings_to_remove))
    # Транслитерация
    df[new_column_name] = df[new_column_name].apply(lambda x: translit(x, 'ru', reversed=True))

    # Удаление лишних пробелов
    df[new_column_name] = df[new_column_name].str.replace(r'\s+',' ', regex=True).str.strip()
    return df




In [6]:
df_1C = process_column(df_1C, 'Наименование', 'Наименование обработанное')
df = process_column(df, 'Наименование', 'Наименование обработанное')

Шины "Роудстоун" транслитерировались в "roudstoun" (оригинальное название "Roadstone"). Шины "Бриджстоун" - в "bridzhstoun" (ориг. название "bridgestone"). Транслитерированные и оригинальные названия не совсем схожи, но для нечеткого сравнения это намного лучше, чем оставлять русские наименования.

In [7]:
df_1C[df_1C['Наименование'].str.contains('сто')].sample(n=3, random_state=0)

,Наименование,Код,Наименование обработанное
2771,R14 185/70 Автошина Роудстоун T 92 WINGUARD WI...,ЦБ-00006744,r14 185/70 roudstoun t 92 winguard winspike xl sh
2737,R14 185/65 Роудстоун Winguard WinSpike XL (шип),ЦБ-00006111,r14 185/65 roudstoun winguard winspike xl ship
3169,R15 195/65 Бриджстоун IC7000S (шип),ЦБ-00006103,r15 195/65 bridzhstoun ic7000s ship


In [8]:
df_1C[df_1C['Наименование'].str.contains('sto')].sample(n=3, random_state=1)

,Наименование,Код,Наименование обработанное
1700,R17 235/65 Bridgestone Turanza ER 30,ЦБ-00003840,r17 235/65 bridgestone turanza er 30
733,R16 205/55 Bridgestone Turanza T001,ЦБ-00002776,r16 205/55 bridgestone turanza t001
2896,R15 185/65 Roadstone Winguard WinSpike (шип),ЦБ-00006251,r15 185/65 roadstone winguard winspike ship


Применение методов нечеткого сравнения token_set_ratio, ratio, WRatio, а также модель "all-distilroberta-v1" для создания эмбеддингов и сравнение с поомщью cos сходства.

In [10]:
# Замер времени выполнения кода
start_time = time.time()

# Поиск наилучшего совпадения и уровня схожести методом token_set_ratio библиотеки rapidfuzz
# Вычисляет расстояние Левенштейна между двумя множествами токенов, игнорирует порядок слов и дубликаты
def get_token_set_ratio(query, choices):
    match = process.extractOne(query, choices, scorer=fuzz.token_set_ratio)
    if match:
        return match[0], match[1]
    else:
        return None, None

best_matches = df['Наименование обработанное'].apply(lambda x: get_token_set_ratio(x, df_1C['Наименование обработанное']))
df['token_set_ratio уровень сходства'] = best_matches.apply(lambda x: x[1])
df['token_set_ratio лучшее совпадение'] = best_matches.apply(lambda x: x[0])

# Поиск наилучшего совпадения и уровня схожести методом ratio библиотеки rapidfuzz
# Вычисляет расстояние Левенштейна, которое представляет собой минимальное количество односимвольных изменений (вставок, удалений или замен), необходимых для преобразования одной строки в другую
def get_ratio(query, choices):
    match = process.extractOne(query, choices, scorer=fuzz.ratio)
    if match:
        return match[0], match[1]
    else:
        return None, None

best_matches = df['Наименование обработанное'].apply(lambda x: get_ratio(x, df_1C['Наименование обработанное']))
df['ratio уровень сходства'] = best_matches.apply(lambda x: x[1])
df['ratio лучшее совпадение'] = best_matches.apply(lambda x: x[0])

# Поиск наилучшего совпадения и уровня схожести методом WRatio библиотеки rapidfuzz
# Использует предыдущие два метода, после чего взвешивает их результаты (т.е. учитывает различные аспекты строк)
def get_Wratio(query, choices):
    match = process.extractOne(query, choices, scorer=fuzz.WRatio)
    if match:
        return match[0], match[1]
    else:
        return None, None

best_matches = df['Наименование обработанное'].apply(lambda x: get_ratio(x, df_1C['Наименование обработанное']))
df['WRatio уровень сходства'] = best_matches.apply(lambda x: x[1])
df['WRatio лучшее совпадение'] = best_matches.apply(lambda x: x[0])



# Поиск лучшего совпадения с помощью sentence_transformers, используя модель all-distilroberta-v1
model = SentenceTransformer('all-distilroberta-v1')
embeddings1 = model.encode(df['Наименование обработанное'].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df_1C['Наименование обработанное'].tolist(), convert_to_tensor=True)

similarities = util.pytorch_cos_sim(embeddings1, embeddings2)
best_match_indices = similarities.argmax(dim=1).cpu().numpy()
df['sentence_transformers уровень сходства'] = similarities.max(dim=1).values.cpu().numpy()
df['sentence_transformers уровень сходства'] = df['sentence_transformers уровень сходства']*100
df['sentence_transformers лучшее совпадение'] = df_1C['Наименование обработанное'].iloc[best_match_indices].values

print(f"Время выполнения функции: {time.time() - start_time} секунд")

Время выполнения функции: 67.12573409080505 секунд


После изучения результатов (будет далее) предыдущих методов было принято решение разработать свой алгоритм для нечеткого сравнения строк.

Алгоритм следующий:

1) Удаляем из текста все, кроме букв и цифр (даже пробелы удаляются).
2) Создаем n-граммы (последовательности из n символов).
3) Одна функция будет создавать последовательности из 1, 2, 3 символов. Вторая - из 1, 2, 3, 4 символов.
4) Затем подсчитываем количество вхождений каждой n-граммы для каждой строки.
5) Применяем косинусное сходство.
6) Выделяем по 2 наилучших сходства.

In [12]:
# Удаляем все кроме букв и цифр
def preprocess_text(text):
    return ''.join(filter(str.isalnum, text.lower()))

# Создаем n-граммы
def create_1_3ngrams(text):
    ngrams = []
    for n in range(1, 4):
        ngrams.extend([text[i:i+n] for i in range(len(text)-n+1)])
    return ngrams
    
def create_1_4ngrams(text):
    ngrams = []
    for n in range(1, 5):
        ngrams.extend([text[i:i+n] for i in range(len(text)-n+1)])
    return ngrams

In [13]:
df['1-3ngrams'] = df['Наименование обработанное'].apply(lambda x: ' '.join(create_1_3ngrams(preprocess_text(x))))
df['1-4ngrams'] = df['Наименование обработанное'].apply(lambda x: ' '.join(create_1_4ngrams(preprocess_text(x))))

df_1C['1-3ngrams'] = df_1C['Наименование обработанное'].apply(lambda x: ' '.join(create_1_3ngrams(preprocess_text(x))))
df_1C['1-4ngrams'] = df_1C['Наименование обработанное'].apply(lambda x: ' '.join(create_1_4ngrams(preprocess_text(x))))

In [14]:
combined_1_4ngrams = pd.concat([df_1C['1-4ngrams'], df['1-4ngrams']])
combined_1_3ngrams = pd.concat([df_1C['1-3ngrams'], df['1-3ngrams']])

# Применение CountVectorizer для подсчета вхождений n-грамм
vectorizer_1_4 = CountVectorizer()
combined_count_vec_1_4 = vectorizer_1_4.fit_transform(combined_1_4ngrams)

vectorizer_1_3 = CountVectorizer()
combined_count_vec_1_3 = vectorizer_1_3.fit_transform(combined_1_3ngrams)


In [15]:
# Нормализация матриц с помощью Normalizer
normalizer_1_3 = Normalizer()
combined_count_vec_1_3_norm = normalizer_1_3.fit_transform(combined_count_vec_1_3)

normalizer_1_4 = Normalizer()
combined_count_vec_1_4_norm = normalizer_1_4.fit_transform(combined_count_vec_1_4)

In [16]:
count_vec_1_3_df_1C_norm = combined_count_vec_1_3_norm[:len(df_1C)]
count_vec_1_3_df_norm = combined_count_vec_1_3_norm[len(df_1C):]

count_vec_1_4_df_1C_norm = combined_count_vec_1_4_norm[:len(df_1C)]
count_vec_1_4_df_norm = combined_count_vec_1_4_norm[len(df_1C):]

In [17]:
cos_matrix_1_3 = count_vec_1_3_df_norm.dot(count_vec_1_3_df_1C_norm.T).toarray()
cos_matrix_1_4 = count_vec_1_4_df_norm.dot(count_vec_1_4_df_1C_norm.T).toarray()

In [18]:
top_2_indices_1_3 = np.argsort(cos_matrix_1_3, axis=1)[:, -2:]
top_2_values_1_3 = np.take_along_axis(cos_matrix_1_3, top_2_indices_1_3, axis=1)
df['Топ1 индекс cos сходства 1_3'] = top_2_indices_1_3[:, -1]
df['Топ2 индекс cos сходства 1_3'] = top_2_indices_1_3[:, -2]
df['Топ1 уровень cos сходства 1_3'] = top_2_values_1_3[:, -1]
df['Топ2 уровень cos сходства 1_3'] = top_2_values_1_3[:, -2]
df['Cos сходство топ1 совпадение 1-3'] = df['Топ1 индекс cos сходства 1_3'].apply(lambda x: df_1C['Наименование обработанное'].iloc[x])
df['Cos сходство топ2 совпадение 1-3'] = df['Топ2 индекс cos сходства 1_3'].apply(lambda x: df_1C['Наименование обработанное'].iloc[x])


top_2_indices_1_4 = np.argsort(cos_matrix_1_4, axis=1)[:, -2:]
top_2_values_1_4 = np.take_along_axis(cos_matrix_1_4, top_2_indices_1_4, axis=1)
df['Топ1 индекс cos сходства 1_4'] = top_2_indices_1_4[:, -1]
df['Топ2 индекс cos сходства 1_4'] = top_2_indices_1_4[:, -2]
df['Топ1 уровень cos сходства 1_4'] = top_2_values_1_4[:, -1]
df['Топ2 уровень cos сходства 1_4'] = top_2_values_1_4[:, -2]
df['Cos сходство топ1 совпадение 1-4'] = df['Топ1 индекс cos сходства 1_4'].apply(lambda x: df_1C['Наименование обработанное'].iloc[x])
df['Cos сходство топ2 совпадение 1-4'] = df['Топ2 индекс cos сходства 1_4'].apply(lambda x: df_1C['Наименование обработанное'].iloc[x])

df['Топ1 уровень cos сходства 1_3'] = df['Топ1 уровень cos сходства 1_3']*100
df['Топ2 уровень cos сходства 1_3'] = df['Топ2 уровень cos сходства 1_3']*100
df['Топ1 уровень cos сходства 1_4'] = df['Топ1 уровень cos сходства 1_4']*100
df['Топ2 уровень cos сходства 1_4'] = df['Топ2 уровень cos сходства 1_4']*100

In [19]:
df.drop(['Топ1 индекс cos сходства 1_4','Топ2 индекс cos сходства 1_4', 'Топ1 индекс cos сходства 1_3','Топ2 индекс cos сходства 1_3','1-3ngrams', '1-4ngrams'], axis=1, inplace=True)

In [20]:
df

,Наименование,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,ratio уровень сходства,ratio лучшее совпадение,WRatio уровень сходства,WRatio лучшее совпадение,sentence_transformers уровень сходства,sentence_transformers лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
0,R13 155/70 RoyalBlack Royal ECO,r13 155/70 royalblack royal eco,96.774194,r13 175/70 royalblack royal eco,96.774194,r13 175/70 royalblack royal eco,96.774194,r13 175/70 royalblack royal eco,97.717796,r13 175/70 royalblack royal eco,92.063492,92.063492,r13 175/70 royalblack royal eco,r13 155/70 royalblack royal a/s,91.621541,90.000000,r13 155/70 royalblack royal a/s,r13 175/70 royalblack royal eco
1,R13 155/70 RoyalBlack ROYALMILE,r13 155/70 royalblack royalmile,87.096774,r13 155/70 royalblack royal a/s,98.412698,r13 155/70 royalblack royal mile,98.412698,r13 155/70 royalblack royal mile,95.148521,r13 155/70 royalblack royal mile,100.000000,91.265332,r13 155/70 royalblack royal mile,r13 155/70 75t royal black royal mile,100.000000,90.131661,r13 155/70 royalblack royal mile,r13 155/70 royalblack royal a/s
2,R13 155/70 Sailun Atrezzo Eco,r13 155/70 sailun atrezzo eco,100.000000,r13 155/70 75t sailun atrezzo eco,93.548387,r13 155/70 75t sailun atrezzo eco,93.548387,r13 155/70 75t sailun atrezzo eco,97.430206,r13 155/70 75t sailun atrezzo eco,87.671401,80.000000,r13 155/70 75t sailun atrezzo eco,r14 185/70 sailun atrezzo eco,85.280287,78.787879,r13 155/70 75t sailun atrezzo eco,r14 175/70 sailun atrezzo eco
3,R13 155/70 Kapsen H202 ComfortMax A/S,r13 155/70 kapsen h202 comfortmax a/s,100.000000,r13 155/70 75t kapsen h202 comfortmax a/s,94.871795,r13 155/70 75t kapsen h202 comfortmax a/s,94.871795,r13 155/70 75t kapsen h202 comfortmax a/s,97.726921,r13 155/70 75t kapsen h202 comfortmax a/s,90.112711,73.929609,r13 155/70 75t kapsen h202 comfortmax a/s,r16 225/70 103t kapsen h202 comfortmax a/s,88.249750,71.269665,r13 155/70 75t kapsen h202 comfortmax a/s,r16 225/70 103t kapsen h202 comfortmax a/s
4,R13 155/70 Kapsen HD918,r13 155/70 kapsen hd918,88.000000,r13 175/70 82t kapsen hd918,88.000000,r13 175/70 82t kapsen hd918,88.000000,r13 175/70 82t kapsen hd918,95.670876,r13 175/70 82t kapsen hd918,71.275096,61.864711,r13 175/70 82t kapsen hd918,r14 185/70 88h kapsen hd918,65.079137,56.907440,r13 175/70 82t kapsen hd918,r14 185/70 88h kapsen hd918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,R21 315/40 ILINK SnowGripper 2,r21 315/40 ilink snowgripper 2,96.551724,r21 315/40 ilink snowgripper ii 115h xl,84.057971,r21 315/40 ilink snowgripper ii 115h xl,84.057971,r21 315/40 ilink snowgripper ii 115h xl,92.165817,r21 315/35 ilink snowgripper ii 111h xl,85.052389,71.993233,r21 315/40 ilink snowgripper ii 115h xl,r21 285/40 ilink snowgripper ii 109h xl,84.460955,69.289103,r21 315/40 ilink snowgripper ii 115h xl,r21 275/40 ilink snowgripper ii 107h xl
2382,R21 315/40 Arivo Winmaster ProX ARW5,r21 315/40 arivo winmaster prox arw5,100.000000,r21 315/40 winmaster prox arw5,90.909091,r21 315/40 winmaster prox arw5,90.909091,r21 315/40 winmaster prox arw5,96.218941,r21 315/40 winmaster prox arw5,85.455020,68.364016,r21 315/40 winmaster prox arw5,r21 275/45 winmaster prox arw5,83.081859,64.907702,r21 315/40 winmaster prox arw5,r21 275/45 winmaster prox arw5
2383,R22 275/40 ILINK SnowGripper 2,r22 275/40 ilink snowgripper 2,96.551724,r22 275/40 ilink snowgripper ii 107h xl,84.057971,r22 275/40 ilink snowgripper ii 107h xl,84.057971,r22 275/40 ilink snowgripper ii 107h xl,91.927757,r22 275/40 ilink snowgripper ii 107h xl,85.104851,74.095857,r22 275/40 ilink snowgripper ii 107h xl,r21 275/40 ilink snowgripper ii 107h xl,84.503108,73.214299,r22 275/40 ilink snowgripper ii 107h xl,r21 275/40 il

In [22]:
df[(df['WRatio лучшее совпадение']!=df['Cos сходство топ1 совпадение 1-3'])|(df['ratio лучшее совпадение']!=df['Cos сходство топ1 совпадение 1-4'])].sample(20, random_state=29)

,Наименование,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,ratio уровень сходства,ratio лучшее совпадение,WRatio уровень сходства,WRatio лучшее совпадение,sentence_transformers уровень сходства,sentence_transformers лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
1252,R21 325/30 Kapsen RS26 PracticalMax HP,r21 325/30 kapsen rs26 practicalmax hp,95.890411,r21 325/30 xl 108w kapsen rs26 practicalmax h/p,90.909091,r20 265/50 kapsen rs26 practical max hp,90.909091,r20 265/50 kapsen rs26 practical max hp,92.199516,r20 265/50 kapsen rs26 practical max hp,86.916383,79.042540,r21 325/30 xl 108w kapsen rs26 practicalmax h/p,r20 265/50 kapsen rs26 practical max hp,85.201287,78.026689,r21 325/30 xl 108w kapsen rs26 practicalmax h/p,r20 265/50 kapsen rs26 practical max hp
46,R14 175/65 Nokian Nordman SX3,r14 175/65 nokian nordman sx3,100.000000,r14 175/65 nokian tyres t 82 nordman sx3,91.228070,r15 195/65 nokian nordman sx,91.228070,r15 195/65 nokian nordman sx,95.181915,r14 175/70 nokian nordman sx2,81.632653,81.508540,r14 175/70 nokian nordman sx2,r14 175/65 nokian tyres t 82 nordman sx3,78.898188,78.571429,r14 175/65 nokian tyres t 82 nordman sx3,r14 175/70 nokian nordman sx2
1725,R16C 195/75 Viatti 107/105R Vettore Inverno V-...,r16c 195/75 viatti 107/105r vettore inverno v 524,100.000000,r16c 195/75 viatti v 524,87.234043,r16c 195/75 viatti107/105 vettore brina v 525,87.234043,r16c 195/75 viatti107/105 vettore brina v 525,95.869583,r16c 205/65 viatti vettore inverno v 524,82.908117,81.116704,r16c 195/75 viatti vettore inverno zima v 524 ...,r16c 195/75 viatti vettore inverno v 524 ship,79.567831,78.884265,r16c 195/75 viatti vettore inverno v 524 ship,r16c 195/75 viatti vettore inverno zima v 524 ...
410,R16 235/70 Nokian Nordman S2 SUV,r16 235/70 nokian nordman s2 suv,95.238095,r16 215/70 nokian nordman s suv,95.238095,r16 215/70 nokian nordman s suv,95.238095,r16 215/70 nokian nordman s suv,92.998634,r16 245/70 nokian nordman s suv,82.707603,82.707603,r16 265/70 nokian nordman s suv,r16 245/70 nokian nordman s suv,77.868134,77.868134,r16 215/70 nokian nordman s suv,r16 225/70 nokian nordman s suv
1477,R15 235/75 Triangle Group TR777 Snow Lion,r15 235/75 triangle group tr777 snow lion,86.363636,r15 235/75 triangle tr258,76.315789,r18 225/55 triangle group snow pl01,76.315789,r18 225/55 triangle group snow pl01,81.163895,r15 205/70 triangle tr777,60.163557,56.911878,r15 235/75 triangle tr258,r15 205/65 triangle tr777,58.585010,51.214752,r15 235/75 triangle tr258,r15 205/65 triangle tr777
790,R18 225/60 Arivo Traverso ARV H/T,r18 225/60 arivo traverso arv h/t,90.000000,r18 225/60 104h xl traverso arv h/t,86.666667,r18 235/60 traverso arv h/t,86.666667,r18 235/60 traverso arv h/t,93.430672,r18 235/60 traverso arv h/t,73.994007,72.549020,r18 255/60 traverso arv h/t,r18 225/60 104h xl traverso arv h/t,69.800230,67.748454,r18 225/60 104h xl traverso arv h/t,r18 255/60 traverso arv h/t
1233,R21 275/40 ILINK L-Zeal 56,r21 275/40 ilink l zeal 56,80.000000,r19 235/45 ilink l zeal56 95w,81.355932,r21 275/50 ilink l zeal56 113v xl,81.355932,r21 275/50 ilink l zeal56 113v xl,90.096024,r21 275/45 ilink l zeal56 110w xl z,75.153823,74.520394,r21 275/50 ilink l zeal56 113v xl,r20 275/40 ilink l zeal56 106w xl z,72.887930,72.168784,r20 275/40 ilink l zeal56 106w xl z,r21 275/50 ilink l zeal56 113v xl
1189,R20 275/40 Arivo Ultra ARZ 5,r20 275/40 arivo ultra arz 5,81.632653,r20 275/45 ultra arz5,81.632653,r20 245/40 ultra arz5,81.632653,r20 245/40 ultra arz5,91.758797,r20 275/45 ultra arz5,73.951790,67.441860,r20 275/45 ultra arz5,r20 275/40 106w xl ultra arz5,70.048626,64.002048,r20 275/45 ultra arz5,r20 275/40 106w xl ultra arz5
1906,R17 225/60 Nokian Hakkapeliitta 8 SUV,r17 225/60 nok

### Как можно заметить, методы ratio, WRatio и sentence_transformers значительно хуже показали себя в сравнении с остальными, поэтому их удалим и сделаем взвешенное решение на основе оставшихся трех методов

In [23]:
df_remaining = df.iloc[:,[1,2,3,-8,-7,-6,-5,-4,-3,-2,-1]].copy()
df_remaining = df_remaining.round(3)

In [24]:
df_remaining.sample()

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
1756,r17 215/45 massimo mas winter79,83.019,r17 215/45 massimo mas winter 79 91 v xl,91.987,86.322,r17 215/45 massimo mas winter 79 91 v xl,r17 225/45 massimo mas winter 79 94 v,91.581,83.761,r17 215/45 massimo mas winter 79 91 v xl,r17 225/45 massimo mas winter 79 94 v


In [25]:
df_remaining[~(df_remaining['Cos сходство топ1 совпадение 1-3']==df_remaining['Cos сходство топ1 совпадение 1-4'])]

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
0,r13 155/70 royalblack royal eco,96.774,r13 175/70 royalblack royal eco,92.063,92.063,r13 175/70 royalblack royal eco,r13 155/70 royalblack royal a/s,91.622,90.000,r13 155/70 royalblack royal a/s,r13 175/70 royalblack royal eco
25,r13 175/70 pirelli formula energy,90.909,r14 175/65 pirelli formula energy,77.358,75.472,r14 175/65 pirelli formula energy,r14 185/60 pirelli formula energy,74.359,74.359,r15 195/50 pirelli formula energy,r14 175/65 pirelli formula energy
33,r14 175/65 royalblack royalmile,87.097,r14 175/65 royalblack royal eco,91.265,90.636,r14 175/65 82t royal black royal mile,r14 175/65 royalblack royal eco,89.630,89.326,r14 175/65 royalblack royal eco,r14 175/65 82t royal black royal mile
35,r14 175/65 ilink l grip 55,80.000,r15 185/65 ilink l grip55 88h,70.198,70.198,r15 205/65 ilink l grip55 94v,r15 185/65 ilink l grip55 88h,68.579,68.579,r15 185/65 ilink l grip55 88h,r16 215/65 ilink l grip55 98h
46,r14 175/65 nokian nordman sx3,100.000,r14 175/65 nokian tyres t 82 nordman sx3,81.633,81.509,r14 175/70 nokian nordman sx2,r14 175/65 nokian tyres t 82 nordman sx3,78.898,78.571,r14 175/65 nokian tyres t 82 nordman sx3,r14 175/70 nokian nordman sx2
...,...,...,...,...,...,...,...,...,...,...,...
2318,r20 255/50 kapsen icemax rw501,77.551,r19 255/50 kapsen rw501 107h xl,58.824,57.191,r19 255/50 kapsen rw501 107h xl,r20 255/50 kapsen aw33 109h xl,55.354,54.795,r20 255/50 kapsen aw33 109h xl,r19 255/50 kapsen rw501 107h xl
2335,r20 275/45 kapsen snowshoes aw33,84.615,r20 275/45 xl kapsen aw33 110h,56.360,52.185,r20 275/45 xl kapsen aw33 110h,r20 275/45 kapsen rw501 110h,51.560,51.168,r20 275/45 kapsen rw501 110h,r20 275/45 xl kapsen aw33 110h
2341,r20 275/45 yokohama ice guard stud ig55,83.582,r16 195/55 yokohama ice guard stud ig55 ship,77.069,77.049,r17 275/65 yokohama iceguard stud ig55 ship,r17 235/45 yokohama iceguard stud ig 65,76.667,75.147,r17 235/45 yokohama iceguard stud ig 65,r17 275/65 yokohama iceguard stud ig55 ship
2342,r20 275/75 pirelli ice zero,85.185,r17 215/50 pirelli ice zero,65.184,64.304,r20 275/40 pirelli s zero,r14 185/65 pirelli ice zero,63.943,63.943,r17 225/45 pirelli ice zero,r14 185/65 pirelli ice zero


In [26]:
df_remaining[~((df_remaining['Cos сходство топ1 совпадение 1-3']==df_remaining['Cos сходство топ1 совпадение 1-4'])|(df_remaining['Cos сходство топ2 совпадение 1-3']==df_remaining['Cos сходство топ1 совпадение 1-4']))]

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
25,r13 175/70 pirelli formula energy,90.909,r14 175/65 pirelli formula energy,77.358,75.472,r14 175/65 pirelli formula energy,r14 185/60 pirelli formula energy,74.359,74.359,r15 195/50 pirelli formula energy,r14 175/65 pirelli formula energy
219,r15 205/70 kapsen h201 touringmax a/s,64.516,r15 205/70 ahiles 122,45.926,45.763,r13 155/70 75t kapsen h202 comfortmax a/s,r16 205/60 kapsen comfortmax as h202,44.331,40.254,r15 205/70 kapsen rw506 100t xl ship,r13 155/70 75t kapsen h202 comfortmax a/s
410,r16 235/70 nokian nordman s2 suv,95.238,r16 215/70 nokian nordman s suv,82.708,82.708,r16 265/70 nokian nordman s suv,r16 245/70 nokian nordman s suv,77.868,77.868,r16 215/70 nokian nordman s suv,r16 225/70 nokian nordman s suv
428,r16 265/70 royalblack royal sport,100.000,r16 265/70 112h royalblack royal sport,92.537,92.537,r16 245/70 royalblack royal sport,r16 225/70 royalblack royal sport,90.625,90.625,r16 235/70 royalblack royal sport,r16 245/70 royalblack royal sport
659,r17 235/45 sailun atrezzo zsr,100.000,r17 235/45 97y sailun atrezzo zsr,88.889,88.889,r18 235/45 sailun atrezzo zsr,r17 225/45 sailun atrezzo zsr,87.879,87.879,r19 235/45 sailun atrezzo zsr,r18 235/45 sailun atrezzo zsr
675,r17 235/65 sunwide conquest,92.000,r18 235/65 sunwide conquest 106h,81.694,81.145,r17 265/65 sunwide conquest 112h,r17 225/65 sunwide conquest 102h,80.013,78.967,r18 235/65 sunwide conquest 106h,r17 265/65 sunwide conquest 112h
678,r17 235/65 massimo stella s2,70.000,r17 235/45 97w massimo ottima p1,47.741,47.717,r17 235/65 winmaster arw2,r14 175/65 massimo mas winter 79 82 t,42.683,42.553,r17 235/45 97w massimo ottima p1,r14 175/65 massimo mas winter 79 82 t
695,r17 245/40 arivo ultra arz 4,77.551,r17 205/40 ultra arz4,65.735,65.735,r18 245/40 ultra arz4,r17 205/40 ultra arz4,60.664,60.583,r17 245/40 95w xl ultra arz5,r18 245/40 ultra arz4
738,r18 215/55 kapsen k3000 papide,92.857,r16 215/55 xl 97w kapsen k3000 papide,77.876,77.876,r16 215/55 xl 97w kapsen k3000 papide,r17 215/55 xl 98w kapsen k3000 papide,75.683,74.247,r18 215/55 kapsen k3000 99w xl z,r16 215/55 xl 97w kapsen k3000 papide
761,r18 225/45 kapsen s2000 headking,93.333,r17 225/45 xl 94w kapsen s2000 headking,78.372,78.248,r18 255/45 xl 103w kapsen s2000 headking,r18 245/45 xl 100w kapsen s2000 headking,74.852,74.852,r17 235/45 kapsen s2000 headking xl 97w,r17 225/45 xl 94w kapsen s2000 headking


In [42]:
df_remaining[(df_remaining['Топ1 уровень cos сходства 1_3']>97)&(df_remaining['Топ1 уровень cos сходства 1_3']<=100)].head(20)

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4
1,r13 155/70 royalblack royalmile,87.097,r13 155/70 royalblack royal a/s,100.0,91.265,r13 155/70 royalblack royal mile,r13 155/70 75t royal black royal mile,100.0,90.132,r13 155/70 royalblack royal mile,r13 155/70 royalblack royal a/s
11,r13 175/70 royalblack royal comfort,100.000,r13 175/70 royalblack royal comfort,100.0,92.958,r13 175/70 royalblack royal comfort,r14 175/70 royalblack royal comfort,100.0,92.157,r13 175/70 royalblack royal comfort,r14 175/70 royalblack royal comfort
12,r13 175/70 royalblack royal eco,100.000,r13 175/70 royalblack royal eco,100.0,92.063,r13 175/70 royalblack royal eco,r14 175/70 royalblack royal eco,100.0,91.111,r13 175/70 royalblack royal eco,r14 175/70 royalblack royal eco
28,r14 175/65 kama breeze nk 132,100.000,r14 175/65 kama h 82 breeze nk 132,100.0,88.372,r14 175/65 kama breeze nk 132,r14 185/65 kama breeze nk 132,100.0,85.714,r14 175/65 kama breeze nk 132,r14 185/65 kama breeze nk 132
32,r14 175/65 viatti strada asimmetrico v 130,100.000,r14 175/65 viatti strada asimmetrico v 130,100.0,92.754,r14 175/65 viatti strada asimmetrico v 130,r14 185/65 viatti strada asimmetrico v 130,100.0,91.089,r14 175/65 viatti strada asimmetrico v 130,r14 185/65 viatti strada asimmetrico v 130
44,r14 175/65 pirelli formula energy,100.000,r14 175/65 pirelli formula energy,100.0,90.566,r14 175/65 pirelli formula energy,r14 185/65 pirelli formula energy,100.0,88.462,r14 175/65 pirelli formula energy,r14 185/65 pirelli formula energy
49,r14 185/60 kama breeze nk 132,100.000,r14 185/60 kama breeze nk 132,100.0,100.000,r14 185/60 kama breeze nk 132,r14 185/60 kama breeze nk 132,100.0,100.000,r14 185/60 kama breeze nk 132,r14 185/60 kama breeze nk 132
52,r14 185/60 viatti strada asimmetrico v 130,100.000,r14 185/60 viatti strada asimmetrico v 130,100.0,92.754,r14 185/60 viatti strada asimmetrico v 130,r14 185/65 viatti strada asimmetrico v 130,100.0,92.079,r14 185/60 viatti strada asimmetrico v 130,r15 185/60 viatti strada asimmetrico v 130
61,r14 185/60 pirelli formula energy,100.000,r14 185/60 pirelli formula energy,100.0,90.566,r14 185/60 pirelli formula energy,r14 185/65 pirelli formula energy,100.0,88.462,r14 185/60 pirelli formula energy,r14 185/65 pirelli formula energy
66,r14 185/65 kama breeze nk 132,100.000,r14 185/65 kama h 86 breeze nk 132,100.0,88.372,r14 185/65 kama breeze nk 132,r14 185/60 kama breeze nk 132,100.0,85.714,r14 185/65 kama breeze nk 132,r14 185/60 kama breeze nk 132


Пристальный анализ дал понять, что нужно выбирать топ1 косинусное сходство в зависимости от разницы между уровнем сходства топ1 и топ2 значения. Если разница между ур. схожести топ1 и топ2 значениями у метрики с 1-4 n-граммами выше, чем таковая у метрики с 1-3 n-граммами, то выбираем первую. Иначе - вторую. И token_set_ratio используем при значениях уровня сходства выше 97.

Создадим взвешенное сходство

In [43]:
df_remaining['Взвешенное сходство'] = None
df_remaining['Уровень взвеш. сходства'] = None

for index, row in df_remaining.iterrows():
    # Если топ1 ур. cos сходства у 1-3 n-грамм больше или равен, чем у token_set_ratio и больше 97, то устанавливаем сходство соответствующее этому значению.
    if row['Топ1 уровень cos сходства 1_3']>=row['token_set_ratio уровень сходства'] and row['Топ1 уровень cos сходства 1_3']>=97:
        df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-3']
        df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3']
        
    # Применяем token_set_ratio, если уровень сходства у него больше 97
    elif row['token_set_ratio уровень сходства']>=97:
        df_remaining.at[index, 'Взвешенное сходство'] = row['token_set_ratio лучшее совпадение']
        df_remaining.at[index, 'Уровень взвеш. сходства'] = row['token_set_ratio уровень сходства']
        
    # Если лучшие косинусные совпадения для 1-3n-грамм и для 1-4n-грамм одинаковы, во взвешенное значение ставим одно из них
    elif row['Cos сходство топ1 совпадение 1-3']==row['Cos сходство топ1 совпадение 1-4']:
        df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-3']
        df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3']
        
    # Если топ1 косинусное совпадение для 1-3n-грамм совпадает с топ2 кос. совпадением для 1-4n-грамм, аналогично в обратную сторону - рассчитываем какая пара выдаст большее
    # среднее значение и устанавливаем наименование ему соответствующее
    elif row['Cos сходство топ1 совпадение 1-3']==row['Cos сходство топ2 совпадение 1-4'] and row['Cos сходство топ2 совпадение 1-3']==row['Cos сходство топ1 совпадение 1-4']:
        mean_cos_sim_top2ngr3_top1ngr4 = (row['Топ2 уровень cos сходства 1_3'] + row['Топ1 уровень cos сходства 1_4']) / 2
        mean_cos_sim_top1ngr3_top2ngr4 = (row['Топ1 уровень cos сходства 1_3'] + row['Топ2 уровень cos сходства 1_4']) / 2
        
        if mean_cos_sim_top2ngr3_top1ngr4 >= mean_cos_sim_top1ngr3_top2ngr4:
            df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-4']
            df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_4']
        else:
            df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-3']
            df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3']
            
    # Иначе     
    else:
        diff_cos_sim_top1ngr4_top2ngr4 = abs(row['Топ1 уровень cos сходства 1_4'] - row['Топ2 уровень cos сходства 1_4'])
        diff_cos_sim_top1ngr3_top1ngr3 = abs(row['Топ1 уровень cos сходства 1_3'] - row['Топ2 уровень cos сходства 1_3'])
        
        # Если разница топ1 и топ2 сходства для 1-4n-грамм больше чем аналогичная разница для 1-3n-грамм
        # устанавливает наименование, соответствующее топ1 совпадению для 1-4n-грамм
        if diff_cos_sim_top1ngr4_top2ngr4 > diff_cos_sim_top1ngr3_top1ngr3:
            df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-4']
            df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_4']
            
        # Если разница топ1 и топ2 сходства нулевая в обоих случаях, то проверяет является ли лучшее совпадение token_set_ratio в списке всех косинусных топ1-2 совпадений.
        # Если является - устанавливает наименование, соответствующее лучшему совпадению token_set_ratio
        # Если нет - устанавливает топ1 совпадение 1-4n-грамм
        elif diff_cos_sim_top1ngr4_top2ngr4==0 and diff_cos_sim_top1ngr3_top1ngr3==0:
            if row['token_set_ratio лучшее совпадение'] in [row['Cos сходство топ1 совпадение 1-4'],row['Cos сходство топ2 совпадение 1-4'],row['Cos сходство топ1 совпадение 1-3'],row['Cos сходство топ2 совпадение 1-3']]:
                df_remaining.at[index, 'Взвешенное сходство'] = row['token_set_ratio лучшее совпадение']
                df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3'] if (row['Топ1 уровень cos сходства 1_3'] >= row['token_set_ratio уровень сходства']) else row['token_set_ratio уровень сходства']
            else:
                df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-4']
                df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3'] if (row['Топ1 уровень cos сходства 1_3'] >= row['Топ1 уровень cos сходства 1_4']) else row['Топ1 уровень cos сходства 1_4']
                
        # Если разница топ1 и топ2 сходства для 1-4n-грамм меньше, чем аналогичная разница для 1-3n-грамм
        else:
            df_remaining.at[index, 'Взвешенное сходство'] = row['Cos сходство топ1 совпадение 1-3']
            df_remaining.at[index, 'Уровень взвеш. сходства'] = row['Топ1 уровень cos сходства 1_3']

### Посмотрим какие взвешенные сходства получаются, когда значения token_set_ratio отличаются от значений косинусных сходств

In [112]:
df_remaining[df_remaining['token_set_ratio лучшее совпадение']!=df_remaining['Cos сходство топ1 совпадение 1-3']].sample(3, random_state=111)

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4,Взвешенное сходство,Уровень взвеш. сходства
513,r17 215/45 royalblack royal explorer 2,91.892,r19 245/45 royalblack royal explorer,92.524,86.667,r17 215/45 royal black royal explorer ii 91w,r19 245/45 royalblack royal explorer,92.233,85.582,r17 215/45 royal black royal explorer ii 91w,r19 245/45 royalblack royal explorer,r17 215/45 royal black royal explorer ii 91w,92.524
362,r16 215/65 yokohama bluearth ae50,92.537,r16 215/60 yokohama blu earth ae01,96.444,87.514,r16 215/65 yokohama blu earth a ae 50,r16 215/60 yokohama bluearth a ae 50,94.495,83.450,r16 215/65 yokohama blu earth a ae 50,r16 215/60 yokohama bluearth a ae 50,r16 215/65 yokohama blu earth a ae 50,96.444
496,r17 205/40 sailun atrezzo zsr,100.000,r17 205/40 84y sailun atrezzo zsr,88.889,87.671,r17 245/40 sailun atrezzo zsr,r17 205/40 84y sailun atrezzo zsr,86.364,85.280,r17 245/40 sailun atrezzo zsr,r17 205/40 84y sailun atrezzo zsr,r17 205/40 84y sailun atrezzo zsr,100.0


Получились следующие случаи, разберем их:
1) Для значения "r17 215/45 royalblack royal explorer 2" метод token_set_ratio подтянул "r19 245/45 royalblack royal explorer", косинусные сходства обоих методов подтянули "r17 215/45 royal black royal explorer ii 91w". Значение у token_set_ratio ошиблось в диаметре колеса (19 вместо 17), и размере (245/45 вместо 215/45). Косинусное сходство сработало без ошибок ("ii" соответствует "2", а параметр "91w" просто не указан в наименовании нашей выгрузки). Взвешенное сходство использовало совпадение косинусного сходства, все верно.
2) Для значения "r16 215/65 yokohama bluearth ae50" метод token_set_ratio подтянул "r16 215/60 yokohama blu earth ae01", косинусные сходства обоих методов подтянули "r16 215/65 yokohama blu earth a ae 50". Значение у token_set_ratio ошиблось в размере (215/60 вместо 215/65) и параметре "ae50", найдя вместо этого значение с параметром "ae01". Косинусное сходство сработало без ошибок. Взвешенное сходство использовало совпадение косинусного сходства, все верно.
3) Для значения "r17 205/40 sailun atrezzo zsr" метод token_set_ratio подтянул "r17 205/40 84y sailun atrezzo zsr", косинусные сходства обоих методов подтянули "r17 245/40 sailun atrezzo zsr" (топ1 значение) и на топ2 поставили значение как и у token_set_ratio. Метод token_set_ratio сработал без ошибок (аналогично первому случаю для косинусного сходства, параметр "84y" просто не указан в наименовании нашей выгрузки). Косинусное сходство для топ1 совпадений ошиблось в размере, но топ2 подтянули верно. Взвешенное сходство опять отработало верно.

Таким образом, взвешенное сходство использует плюсы всех методов и выбирает наилучший метод для конкретного случая.

In [170]:
final_df = df_remaining.iloc[:,[0,-2,-1]].copy()

In [171]:
final_df = final_df.merge(df_1C.iloc[:,1:3], left_on='Взвешенное сходство', right_on='Наименование обработанное', suffixes=('','_1C'), how='left').drop('Наименование обработанное_1C', axis=1)

In [172]:
final_df = final_df.sort_values(by=['Взвешенное сходство']).reset_index(drop=True)

In [173]:
final_df

,Наименование обработанное,Взвешенное сходство,Уровень взвеш. сходства,Код
0,r13 155/70 kapsen h202 comfortmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,100.0,ЦБ-00007311
1,r15 235/75 kapsen h201 touringmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,38.468,ЦБ-00007311
2,r13 155/70 sailun atrezzo eco,r13 155/70 75t sailun atrezzo eco,100.0,ЦБ-00006440
3,r13 155/70 roadx rxfrost wh12,r13 155/70 roadx rxfrost wh12 75t,100.0,ЦБ-00007708
4,r13 155/70 royalblack royal winter hp,r13 155/70 royalblack royal a/s,85.526,ЦБ-00005134
...,...,...,...,...
2396,r22 285/45 ilink speedking 07,r22 285/45 ilink speedking 07 114v xl,100.0,ЦБ-00007893
2397,r22 285/45 kapsen rs26 practicalmax hp,r22 285/45 xl 114w kapsen rs26 practicalmax h/p,87.287,ЦБ-00007418
2398,r22 295/40 kapsen rs26 practicalmax hp,r22 295/40 xl 112w kapsen rs26 practicalmax h/p,87.287,ЦБ-00007417
2399,r22 295/40 sunwide rs one,r22 295/40z sunwide rs one 112w xl,79.183,ЦБ-00007568


Количество строк в нашей выгрузке немного увеличилось. Произошло это по причине того, что в выгрузке из 1C некоторые значения присутствовали несколько раз и имели несколько кодов.

In [174]:
final_df[final_df.duplicated(subset=['Наименование обработанное'], keep=False)]

,Наименование обработанное,Взвешенное сходство,Уровень взвеш. сходства,Код
97,r14 185/60 kama breeze nk 132,r14 185/60 kama breeze nk 132,100.0,00-00000066
98,r14 185/60 kama breeze nk 132,r14 185/60 kama breeze nk 132,100.0,00-00000786
217,r15 185/60 kama euro hk 236,r15 185/60 kama evro 236,73.08,00-00000808
218,r15 185/60 kama euro hk 236,r15 185/60 kama evro 236,73.08,00-00000122
376,r15 195/65 kama irbis 505,r15 195/65 kama 505 irbis,100.0,00-00000862
377,r15 195/65 kama irbis 505,r15 195/65 kama 505 irbis,100.0,00-00003403
378,r15 195/65 kama breeze nk 132,r15 195/65 kama breeze nk 132,100.0,00-00000861
379,r15 195/65 kama breeze nk 132,r15 195/65 kama breeze nk 132,100.0,00-00000197
380,r15 195/65 kama euro hk 129,r15 195/65 kama evro 129,73.08,00-00000224
381,r15 195/65 kama euro hk 129,r15 195/65 kama evro 129,73.08,00-00000863


В выгрузке 1С содержатся далеко не все значения соответствующие значениями нашей выгрузке, поэтому, разумеется, будут и значения взвешенных сходств, одинаковые для разных наименований:

In [175]:
final_df[final_df.duplicated(subset=['Взвешенное сходство'], keep=False)]

,Наименование обработанное,Взвешенное сходство,Уровень взвеш. сходства,Код
0,r13 155/70 kapsen h202 comfortmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,100.0,ЦБ-00007311
1,r15 235/75 kapsen h201 touringmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,38.468,ЦБ-00007311
4,r13 155/70 royalblack royal winter hp,r13 155/70 royalblack royal a/s,85.526,ЦБ-00005134
5,r13 155/70 royalblack royal eco,r13 155/70 royalblack royal a/s,91.622,ЦБ-00005134
9,r13 155/70 kapsen hd918,r13 175/70 82t kapsen hd918,71.275,ЦБ-00007569
...,...,...,...,...
2366,r21 275/45 arivo ultra arz 5,r21 275/45 ultra arz5,79.43,ЦБ-00003728
2389,r22 275/40 ilink speedking 07,r22 265/40 ilink speedking 07 106v xl,80.48,ЦБ-00007892
2390,r22 265/40 ilink speedking 07,r22 265/40 ilink speedking 07 106v xl,100.0,ЦБ-00007892
2395,r20 275/45 ilink speedking 07,r22 285/45 ilink speedking 07 114v xl,70.969,ЦБ-00007893


Возьмем 10 случайных строк и посмотрим как хорошо отработал наш алгоритм:

In [176]:
final_df.sample(10, random_state=29)

,Наименование обработанное,Взвешенное сходство,Уровень взвеш. сходства,Код
1474,r17 245/40 sailun atrezzo zsr,r17 245/40 sailun atrezzo zsr,100.0,00-00000589
1557,r18 225/40 rydanz roadster r02,r18 225/40 rydanz roadster r02 xl 92w,100.0,ЦБ-00007274
565,r16 205/55 arivo premio arzero,r16 205/55 premio arzero,100.0,ЦБ-00004027
1093,r17 215/50 sailun ice blazer wst3,r17 215/50 sailun ice blazer wst3 95t ship,100.0,ЦБ-00006756
621,r16 205/60 sailun ice blazer wst3,r16 205/60 sailun ice blazer wst3 96t ship,100.0,ЦБ-00006899
444,r15 205/70 royalblack royal ice,r15 205/70 royalblack royal ice 96s,100.0,ЦБ-00007212
238,r15 185/65 massimo ottima p1,r15 185/65 88h massimo ottima p1,100.0,ЦБ-00007239
380,r15 195/65 kama euro hk 129,r15 195/65 kama evro 129,73.08,00-00000224
115,r14 185/60 yokohama ice guard ig50+,r14 185/60 yokohama ice guard ig50,100.0,00-00000080
1081,r17 205/50 nokian hakkapeliitta 8,r17 215/50 nokian hakkapeliitta 8 ship,84.419,ЦБ-00004144


Все наименования идеально подтянулись. Можно проверить последние 2 наименования:

In [177]:
df_remaining[df_remaining['Наименование обработанное'].isin(['r17 205/50 nokian hakkapeliitta 8','r14 185/60 yokohama ice guard ig50+'])]

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4,Взвешенное сходство,Уровень взвеш. сходства
1320,r14 185/60 yokohama ice guard ig50+,98.551,r14 185/60 yokohama ice guard ig50,100.000,87.936,r14 185/60 yokohama ice guard ig50,r14 185/60 yokohama iceguard stud ig55,100.000,85.934,r14 185/60 yokohama ice guard ig50,r14 185/60 yokohama iceguard stud ig55,r14 185/60 yokohama ice guard ig50,100.0
1754,r17 205/50 nokian hakkapeliitta 8,90.141,r17 215/50 nokian hakkapeliitta 8 ship,84.419,77.791,r17 215/50 nokian hakkapeliitta 8 ship,r16 195/50 nokian hakkapeliitta r2,82.353,76.743,r17 215/50 nokian hakkapeliitta 8 ship,r16 195/50 nokian hakkapeliitta r2,r17 215/50 nokian hakkapeliitta 8 ship,84.419


Подтянулись наилучшие варианты наименований, все методы выбрали одинаковые варианты. И ручной поиск в выгрузке 1C подтвердил это, другие похожие наименования в ней отсутствуют.

Теперь посмотрим какой процент совпадений с уровнем схожести в 100 баллов у нас получился

In [178]:
percentage = (final_df[final_df['Уровень взвеш. сходства']==100].shape[0]/final_df.shape[0])*100
print(f"Процент строк с уровнем взвешенного сходства равным 100: {percentage:.1f}%")

Процент строк с уровнем взвешенного сходства равным 100: 53.2%


### Сравнение с таблицей, где сотрудник подтягивал наименования из 1С вручную

In [204]:
df_for_test = pd.read_excel('Подтянутые коды сотрудником.xlsx')

In [205]:
df_for_test

,Наименование,Наименование в 1С,Код в 1С
0,R13 155/70 RoyalBlack Royal ECO,NaN,NaN
1,R13 155/70 RoyalBlack ROYALMILE,R13 155/70 75T ROYAL BLACK ROYAL MILE,ЦБ-00007482
2,R13 155/70 Sailun Atrezzo Eco,R13 155/70 75T Sailun ATREZZO ECO,ЦБ-00006440
3,R13 155/70 Kapsen H202 ComfortMax A/S,R13 155/70 75T KAPSEN H202 ComfortMax A/S,ЦБ-00007311
4,R13 155/70 Kapsen HD918,NaN,NaN
...,...,...,...
2381,R21 315/40 ILINK SnowGripper 2,R21 315/40 ILINK SNOWGRIPPER II 115H XL,ЦБ-00007605
2382,R21 315/40 Arivo Winmaster ProX ARW5,R21 315/40 Winmaster ProX ARW5,00-00000002
2383,R22 275/40 ILINK SnowGripper 2,R22 275/40 ILINK SNOWGRIPPER II 107H XL,ЦБ-00007606
2384,R22 275/40 ILINK WinterVorhut Stud 2,R22 275/40 ILINK WINTERVORHUT STUD Ⅱ 107T XL,ЦБ-00007701


Оставим только строки, в которых есть код из 1C.

Так же применим к наименованиям в этой таблице функции по очистке текста, которые мы применяли для своих датафреймов.

In [206]:
df_for_test_len_befor_delete_na = df_for_test.shape[0]
df_for_test = df_for_test[df_for_test['Код в 1С'].notna()].copy()
df_for_test = df_for_test.rename(columns={'Код в 1С':'Код в 1С_test'})
df_for_test = process_column(df_for_test, 'Наименование', 'Наименование обработанное_test')
df_for_test = process_column(df_for_test, 'Наименование в 1С', 'Наименование в 1С обработанное_test')
df_for_test.drop_duplicates(['Наименование','Наименование в 1С','Код в 1С_test'], inplace=True)
print(f"Количество наименований, отсутствующих в 1С выгрузке: {df_for_test_len_befor_delete_na - df_for_test.shape[0]}")

Количество наименований, отсутствующих в 1С выгрузке: 546


Теперь посмотрим какой процент у нас совпадает с наименованиями выставленными в ручную сотрудником.

Подтянем значения, ключом будет наименование

In [207]:
final_df_merged = final_df.merge(df_for_test.iloc[:,-3:], left_on='Наименование обработанное', right_on='Наименование обработанное_test',how='left')
final_df_merged

,Наименование обработанное,Взвешенное сходство,Уровень взвеш. сходства,Код,Код в 1С_test,Наименование обработанное_test,Наименование в 1С обработанное_test
0,r13 155/70 kapsen h202 comfortmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,100.0,ЦБ-00007311,ЦБ-00007311,r13 155/70 kapsen h202 comfortmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s
1,r15 235/75 kapsen h201 touringmax a/s,r13 155/70 75t kapsen h202 comfortmax a/s,38.468,ЦБ-00007311,NaN,NaN,NaN
2,r13 155/70 sailun atrezzo eco,r13 155/70 75t sailun atrezzo eco,100.0,ЦБ-00006440,ЦБ-00006440,r13 155/70 sailun atrezzo eco,r13 155/70 75t sailun atrezzo eco
3,r13 155/70 roadx rxfrost wh12,r13 155/70 roadx rxfrost wh12 75t,100.0,ЦБ-00007708,ЦБ-00007708,r13 155/70 roadx rxfrost wh12,r13 155/70 roadx rxfrost wh12 75t
4,r13 155/70 royalblack royal winter hp,r13 155/70 royalblack royal a/s,85.526,ЦБ-00005134,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2396,r22 285/45 ilink speedking 07,r22 285/45 ilink speedking 07 114v xl,100.0,ЦБ-00007893,ЦБ-00007893,r22 285/45 ilink speedking 07,r22 285/45 ilink speedking 07 114v xl
2397,r22 285/45 kapsen rs26 practicalmax hp,r22 285/45 xl 114w kapsen rs26 practicalmax h/p,87.287,ЦБ-00007418,ЦБ-00007418,r22 285/45 kapsen rs26 practicalmax hp,r22 285/45 xl 114w kapsen rs26 practicalmax h/p
2398,r22 295/40 kapsen rs26 practicalmax hp,r22 295/40 xl 112w kapsen rs26 practicalmax h/p,87.287,ЦБ-00007417,ЦБ-00007417,r22 295/40 kapsen rs26 practicalmax hp,r22 295/40 xl 112w kapsen rs26 practicalmax h/p
2399,r22 295/40 sunwide rs one,r22 295/40z sunwide rs one 112w xl,79.183,ЦБ-00007568,ЦБ-00007568,r22 295/40 sunwide rs one,r22 295/40z sunwide rs one 112w xl


Удалим строки с пустыми подтянутыми значениями - это те, по которым сотрудник не нашел кода в 1C и мы их удалили из df_for_test

In [208]:
final_df_merged = final_df_merged[final_df_merged['Код в 1С_test'].notna()]

In [239]:
percentage = ((final_df_merged[(final_df_merged['Взвешенное сходство']==final_df_merged['Наименование в 1С обработанное_test'])].shape[0])/final_df_merged.shape[0])*100
print(f"Процент наименований моего алгоритма, совпавших с наименованиями, проставленными сотрудником: {percentage:.1f}%. \n")

Процент наименований моего алгоритма, совпавших с наименованиями, проставленными сотрудником: 93.3%. 



Значения с различающимися наименованиями из 1C:

In [220]:
final_df_merged[~(final_df_merged['Взвешенное сходство']==final_df_merged['Наименование в 1С обработанное_test'])].iloc[:,[0,1,-1]].reset_index(drop=True)

,Наименование обработанное,Взвешенное сходство,Наименование в 1С обработанное_test
0,r13 155/70 royalblack royalmile,r13 155/70 royalblack royal mile,r13 155/70 75t royal black royal mile
1,r13 175/70 kama euro 519,r13 175/70 kama t 82 evro 519 sh,r14 175/70 kama euro 129
2,r14 175/65 kama euro hk 129,r14 175/65 kama euro 129,r14 175/65 kama h 82 evro 129
3,r14 175/65 pirelli formula ice,r14 175/65 pirelli formula energy,r14 175/70 pirelli formula ice ship
4,r14 175/70 kama euro hk 129,r14 175/70 kama euro 129,r14 175/70 kama h 84 evro 129
...,...,...,...
120,r20 275/40 ilink l zeal 56,r20 275/40 ilink l zeal56 106w xl z,r20 275/45 ilink l zeal56 110v xl
121,r20 275/40 sailun atrezzo zsr suv,r20 275/40z 106y sailun atrezzo zsr suv,r20 275/45z 110y sailun atrezzo zsr suv
122,r20 315/35 sailun atrezzo zsr suv,r20 315/35 sailun atrezzo zsr suv,r20 315/35 110y sailun atrezzo zsr suv
123,r21 275/40 arivo ultra arz 5,r21 275/45 ultra arz5,r21 275/40 107w xl ultra arz5


125 наименований различаются

##### Но что можно заметить: мой алгоритм сработал лучше, чем ручная подстановка сотрудником. 
1) В 0 строке мое сходство 100%, у значения сотрудника же оно отличается на значение "75t"
2) В 1 строке у значения сотрудника неверен диаметр колеса
3) Во 2 строке все идентично, разница в параметрах (которые не указаны в нашей выгрузке), в 1С было несколько аналогичных значений
4) В 3 строке у значения сотрудника неверен размер шины
5) В 4 строке так же все идентично, разница в параметрах (которые не указаны в нашей выгрузке), в 1С было несколько аналогичных значений
6) В 120 строке все идентично, разница в параметрах (которые не указаны в нашей выгрузке), в 1С было несколько аналогичных значений
7) В 121 строке аналогичная ситуация
8) В 122 строке аналогичная ситуация
9) В 123 строке мой метод ошибся в размере шины (45 вместо 40)
10) В 124 строке у значения сотрудника неверен диаметр колеса, мой метод отработал верно

Таким образом из рассмотренных наименований мой метод ошибся 1 раз, но показал лучшие совпадения, чем у сотрудника, 4 раза, остальные 5 раз результаты "even" (на одном уровне), то есть невозможно выявить лучшего кандидата, из-за отсутствия параметров в нашей выгрузке

Это еще раз подтверждает, что человеческий фактор при длительной ручной работе дает о себе знать, в этих случаях лучше использовать такие алгоритмы как разработанный мной.

Рассмотрим единственное неверное совпадение, полученное моим алгоритмом

In [222]:
df_remaining[df_remaining['Наименование обработанное']=='r21 275/40 arivo ultra arz 5']

,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4,Взвешенное сходство,Уровень взвеш. сходства
1234,r21 275/40 arivo ultra arz 5,81.633,r21 275/45 ultra arz5,73.952,67.442,r21 275/45 ultra arz5,r21 275/40 107w xl ultra arz5,70.049,64.002,r21 275/45 ultra arz5,r21 275/40 107w xl ultra arz5,r21 275/45 ultra arz5,73.952


Видно, что все три метода, token_set_ratio и cos сходства, подтянули именно значение с неверным размером шины. Косинусное сходство поставило верный вариант на топ2 уровень сходства. Но в данном случае сложно было бы поставить это наименование на первое место, так как 7 дополнительных символов "107w xl" (и особенное влияние здесь оказали цифры, "107") в выгрузке из 1C значительно уменьшают возможность верного сопоставления.

Если бы у нас были столбцы со значениями диаметра колеса и размера шин, то можно было бы сделать следующий алгоритм:

1) Полное сопоставление значений диаметра колеса и размера шин
2) Получение сходства,где параметры колес полностью идентичны, а нечеткое сравнение выполняется в окне полностью совпадающих параметров (это бы, к тому же, ускорило бы работу алгоритма)
3) Рассчет взвешенного сходства

У нас этих столбцов нет, но для решения этой проблемы можно было бы "вытаскивать" диаметр колеса и размер шины из первых двух слов наименования. Но подробное изучение выгрузок дало понять, что это невозможно, так как не обязательно, что эти значения будут находиться в первых двух словах. Также у этих значений бывает самый разный формат в разных выгрузках. Здесь можно было бы делать нечеткое сравнение в окне полного сопоставления диаметра и размера только в случае, если они (диаметр и размер) верного формата, а для остальных случаев использовать нечеткое сравнение без такого окна. Я бы это реализовал, если бы эта задача настоящей, а не кейсовой. В данном же кейсе я хочу сохранить созданный мной алгоритм для дальнейшего широкого использования, с возможностью его корректировки под конкретную задачу.

Или же можно было бы на размеченных данных обучить нейронную сеть или другие алгоритмы машинного обучения. Но, как видно из сопоставлений сотрудником, он делал довольно много ошибок, ошибаясь намного чаще, чем мой алгоритм. И как известно, размечать данные это дорого, при том что для каждой новой задачи размечать данные нужно заново. Мой алгоритм потребует (если вообще потрубет) лишь небольшой корректировки, в зависимости от наличия дополнительных столбцов, на основе которых будут создаваться окна полного сопоставления. В основном он будет работать хорошо и без корректировки.

#### Таким образом, был создан алгоритм взвешенного нечеткого сопоставления, которые использует методы token_set_ratio (рассчет расстояния Левенштейна между двумя множествами токенов, при игнорировании порядка слов и дубликатов) и косинусное сходство различных n-грамм

#### Данный алгоритм показал высокий результат, который находится на одном уровне с длительным (сопровожденным неточностями) ручным сопоставлением.

#### Еще один важный плюс моего алгоритма - скорость работы. Я использовал матричное скалярное произведение csr-матриц для рассчета косинусного сходства, а матричное скалярное произведение очень хорошо оптимизировано в numpy и выполняется очень быстро. Основная же масса известных методов нечеткого сравнения занимает относительно много времени при работе с большими массивами данных.

#### Идея создания такого алгоритма через n-граммы пришла после того, как я заметил, что существующие и мной известные методы в основной массе не очень хорошо справляются с задачей нечеткого сопоставлений. Изначально я просто хотел протестировать этот алгоритм и больших надежд не питал, но он оказался очень хорош, особенно взвешенная версия.

________________
______________
__________

## Дополнение:
## Я создал класс в отдельном модуле для этого алгоритма, теперь его легко можно использовать в 2 строки кода

In [1]:
from WeighedFuzzyNickAlt import WeighedFuzzyTextProcessor

Загрузим заново выгрузки, чтоб отменить все примененные к ним изменения в предыдущем коде

In [5]:
df = pd.read_csv('Gr_Krasn.csv',encoding='Windows-1251', low_memory=False)

In [6]:
df_1C = pd.read_excel('Данные из 1C.xlsx')

In [7]:
processor = WeighedFuzzyTextProcessor(df, df_1C)
result_df = processor.calculate_similarity()

In [261]:
result_df.sample(random_state=229)

,Наименование,Наименование обработанное,token_set_ratio уровень сходства,token_set_ratio лучшее совпадение,Топ1 уровень cos сходства 1_3,Топ2 уровень cos сходства 1_3,Cos сходство топ1 совпадение 1-3,Cos сходство топ2 совпадение 1-3,Топ1 уровень cos сходства 1_4,Топ2 уровень cos сходства 1_4,Cos сходство топ1 совпадение 1-4,Cos сходство топ2 совпадение 1-4,Взвешенное сходство,Уровень взвеш. сходства
2129,R18 245/40 Rydanz Nordica NR01,r18 245/40 rydanz nordica nr01,100.0,r18 245/40 rydanz nordica nr01 97v xl,91.132,81.833,r18 245/40 rydanz nordica nr01 97v xl,r18 225/40 rydanz nordica nr01 92v xl,90.862,79.344,r18 245/40 rydanz nordica nr01 97v xl,r18 225/40 rydanz nordica nr01 92v xl,r18 245/40 rydanz nordica nr01 97v xl,100.0


Все отлично работает

Еще хочу заметить, что в класс добавлены docstring'и, то есть грубо говоря документация класса. Можно нажать с помощью shift+tab (в зависимости от используемой программы) и посмотреть нужную информацию, в том числе в каждом реализованном методе класса.

Также можно использовать отдельно методы класса и для обычного текста. Вот, например, реализация разделения на 1-3 n-граммы слова "текст"

In [15]:
processor.create_ngrams('текст', (1,3))

['т', 'е', 'к', 'с', 'т', 'те', 'ек', 'кс', 'ст', 'тек', 'екс', 'кст']